# Assignment 1: K-Nearest Neighbors Classification (15 marks)

Student Name: Kaixun Yang

Student ID: 

## General info

<b>Due date</b>: Friday, 19 March 2021 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 15% of mark for class. 

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/105477/pages/python-and-jupyter-notebooks?module_item_id=2613813) on Canvas (under Modules> Coding Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. Deep learning libraries such as keras and pytorch are also allowed.  You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  


<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it. We reserve the right to deduct up to 2 marks for unreadable or exessively inefficient code.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board (Piazza -> Assignment_1); we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. Please check the <a href="https://canvas.lms.unimelb.edu.au/courses/105477/modules">CIS Academic Honesty training</a> for more information. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

## Overview

In this assignment, you will implement the K-nearest neighbor (KNN) classification algorithm and apply it to a real-world machine learning data set. In particular we will classify zoo animals into seven animal categories. 

Firstly, you will read in the dataset into a train and a test set (Q1). Secondly, you will implement different distance functions (Q2). Thirdly, you will implement a KNN classifier (Q3). You will apply and evaluate your classifier on the data set exploring different parameters (Q4, Q5). Finally, you will critically discuss your results (Q6).

### Question 1: Loading the data (2.0 marks)

**Instructions:** For this assignment we will develop a K-Nearest Neighbors (KNN) classifier to classify animals in the zoo into pre-defined categories of animals, namely
```
1: mammal
2: bird
3: reptile
4: fish
5: amphibian
6: insect
7: invertebrate
```

We use a data set of zoo animals from the UCI Machine learning repository.

The original data can be found here: https://archive.ics.uci.edu/ml/datasets/Zoo. 

The dataset consists of 101 instances. Each instance corresponds to an animal which has a unique identifier (the name of the animal; first field) and is characterized with 16 features:

```
   1. hair		    Boolean
   2. feathers		Boolean
   3. eggs		    Boolean
   4. milk		    Boolean
   5. airborne		Boolean
   6. aquatic		Boolean
   7. predator		Boolean
   8. toothed		Boolean
   9. backbone		Boolean
  10. breathes		Boolean
  11. venomous		Boolean
  12. fins		    Boolean
  13. legs		    Numeric (set of values: {0,2,4,5,6,8})
  14. tail		    Boolean
  15. domestic		Boolean
  16. catsize		Boolean
```

You need to first obtain this dataset, which is on Canvas (assignment 1). The files we will be using are called *zoo.data* and *zoo.labels*. Make sure the files are saved in the same folder as this notebook.

Both files are in comma-separated value format.

*zoo.features* contains 101 instances, one line per instance. The first field is the unique instance identifier (name of animals). The following fields contain the 16 features, as described above.

*zoo.labels* contains the gold labels (i.e., one of the seven classes above), for one instance per line. Again, the first field is the instance identifier, and the second field the instance label.

**Task**: Read the two files  
1. create a **training_feature** set (list of features for the first 90 instances in the zoo.* files) and a **training_label** set (list of labels for the corresponding). 
2. create a **test_feature** set (list of features of the remaining instances in the zoo.* files) and a **test_label** set (list of labels for the corresponding). 

**Check**: Use the assertion statements in *"For your testing"* to validate your code.

In [96]:
data = open("zoo.features", 'r').readlines()
gold = open("zoo.labels", 'r').readlines()

train_features = []
train_labels   = []
test_features = []
test_labels   = []

###########################
## YOUR CODE BEGINS HERE
###########################


for train_instance in data[:90]:
    train_instance = train_instance.strip()
    train_instance = train_instance.split(",")
    #Convert the datatype from String to Integer
    train_instance_int = list(map(eval, train_instance[1:]))
    train_features.append(train_instance_int)

for test_instance in data[90:]:
    test_instance = test_instance.strip()
    test_instance = test_instance.split(",")
    test_instance_int = list(map(eval, test_instance[1:]))
    test_features.append(test_instance_int)

for train_label in gold[:90]:
    train_label = train_label.strip()
    train_label = train_label.split(",")
    label = int(train_label[1])
    train_labels.append(label)

for test_label in gold[90:]:
    test_label = test_label.strip()
    test_label = test_label.split(",")
    label = int(test_label[1])
    test_labels.append(label)
    
    
###########################
## YOUR CODE BEGINS HERE
###########################



<b>For your testing</b>

In [97]:
assert len(train_features) == len(train_labels)
assert len(train_features[0])==len(train_features[-1])
assert train_features[2][12]==0 and train_labels[2]==4

### Question 2: Distance Functions (2.0 marks)

<b>Instructions</b>: Implement the four distance functions specified below. Use <b>only</b> the library imported below, i.e., <b>do not</b> use implementations from any other Python library. 

1. Eucledian distance
2. Cosine distance
3. Hamming distance 
4. Jaccard distance 

Each distance function takes as input
- Two feature vectors

and returns as output
- The distance between the two feature vectors (float)

**Note** for the purpose of this assignment we consider the numeric feature (legs) to be discretized already with each individual value belonging constituting a separate class.

In [98]:
import math

#########################
# Your answer BEGINS HERE
#########################
#The processing of feature (legs) is in Q5, here are the implementations of four distance functions for common situations.

def eucledian_distance(fv1, fv2):
    # insert code here
    total = 0
    for i in range(len(fv1)):
        total += (fv1[i]-fv2[i])*(fv1[i]-fv2[i])
    return math.sqrt(total)

def cosine_distance(fv1, fv2):
    # insert code here
    dot_product = 0
    square_sum_a = 0
    square_sum_b = 0
    for i in range(len(fv1)):
        dot_product += fv1[i]*fv2[i]
        square_sum_a += fv1[i]*fv1[i]
        square_sum_b += fv2[i]*fv2[i]
    return 1 - dot_product/(math.sqrt(square_sum_a)*math.sqrt(square_sum_b))

def hamming_distance(fv1, fv2):
    # insert code here
    res = 0
    for i in range(len(fv1)):
        if fv1[i] != fv2[i]:
            res = res + 1
    return float(res)
    
def jaccard_distance(fv1, fv2):
    # insert code here
    intersection = 0
    fv1_count = 0
    fv2_count = 0
    for i in range(len(fv1)):
        if fv1[i] == fv2[i] and fv1[i] != 0:
            intersection += 1
        if fv1[i] != 0:
            fv1_count += 1
        if fv2[i] != 0:
            fv2_count += 1
    return 1-intersection/(fv1_count+fv2_count-intersection)


#########################
# Your answer ENDS HERE
#########################



<b>For your testing: </b>

In [99]:
assert round(eucledian_distance([1,0],[0.5,1]),2)==1.12 
assert jaccard_distance([1,1,1,1], [0,1,0,0])==0.75

### Question 3: KNN Classifier (4.0 marks)

<b>Instructions</b>: Here, you implement your KNN classifier. It takes as input 
- training data features
- training data labels
- test data features
- parameter K
- distance function(s) based on which nearest neighbors will be identified

It returns as output 
- the predicted labels for the test data

**Voting stragegy** Your classifier will use majority voting (i.e., no distance weighting)

**You should implement the classifier from scratch yourself**, i.e., <b> you must not</b> use an existing implementation in any Python library.

**Ties**. Ties may occur when computing the K nearest neighbors, or when predicting the class based on the neighborhood. You may deal with ties whichever way you want (as long as you still use the requested distance functions).

In [100]:
def KNN(train_features, train_labels, test_features, k, dist_fun):
    
    predictions = []
    
    ###########################
    ## YOUR CODE BEGINS HERE
    ###########################
    
    import numpy as np
    import random
    for test_feature in test_features:
        distances = []
        results = []
        vote_map = {}
        k_distance_index = []
        temp_distance_index = []
        random_distance_index = []
        max_vote = 0
        
        #Calculate the distance between test_feature and each train_feature
        for train_feature in train_features:
            distances.append(dist_fun(train_feature, test_feature))
            
        #Get the list of orignal index by sort the value of element(distance)
        np_distances = np.array(distances)
        sort_index = np.argsort(np_distances)
        
        #Deal with the ties when computing the K nearest neighbors by random
        for i in range(len(np_distances)):
            if i < k:
                temp_distance_index.append(sort_index[i])
            elif np_distances[sort_index[i]] == np_distances[sort_index[i - 1]]:
                random_distance_index.append(sort_index[i])
            else:
                break;
        if len(random_distance_index) == 0:
            k_distance_index.extend(temp_distance_index)
        else:
            new_distance_index = []
            for index in temp_distance_index:
                if np_distances[index] == np_distances[random_distance_index[0]]:
                    random_distance_index.append(index)
                else:
                    new_distance_index.append(index)
            append_distance_index = random.sample(random_distance_index, k-len(new_distance_index))
            new_distance_index.extend(append_distance_index)
            k_distance_index.extend(new_distance_index)
            
        #Compute the vote of each posssible class
        for distance_index in k_distance_index:
            if train_labels[distance_index] in vote_map:
                vote_map[train_labels[distance_index]] += 1
            else:
                vote_map[train_labels[distance_index]] = 1
                
        #Deal with the ties when when predicting the class based on the neighborhood by random
        labels = sorted(vote_map.items(), key=lambda x: x[1], reverse=True)
        for l in labels:
            if max_vote <= l[1]:
                max_vote = l[1]
                results.append(l[0])
            else:
                break;
        result = random.choice(results)
        predictions.append(result)


    ###########################
    ## YOUR CODE BEGINS HERE
    ###########################
        
    return predictions       

<b>For your testing:</b>

In [101]:
assert KNN([[1,1],[5,5],[1,2]], [1,0,1], [[1,1]], 1, eucledian_distance) == [1]

### Question 4: Evaluation (1.0 marks)
**Instructions:** Write a function that computes the "accuracy" of your classifier. Given a set of predicted lables and a set of gold labels, it returns the fraction of correct labels over all predicted labels. 

**Example**: `The gold truth labels for four test instances are [1, 1, 1, 1]. A system predicted the labels [0, 1, 0, 0] for the same 4 instances. The accuracy of the system is 1/4 = 0.25
`

Your function will take as input 
- gold labels
- predicted labels

It returns as output
- the accuracy value (float).


In [102]:
def accuracy(predict, gold):
    
    ###########################
    ## YOUR CODE BEGINS HERE
    ###########################
    
    
    right_count = 0
    for i in range(len(predict)):
        if predict[i] == gold[i]:
            right_count += 1
    accuracy = right_count/len(predict)


    ###########################
    ## YOUR CODE BEGINS HERE
    ###########################

    return accuracy

<b>For your testing:</b>

In [103]:
assert accuracy([1, 1, 1, 1], [0, 1, 0, 1])==0.5

### Question 5: Applying your KNN classifiers to the Zoo Dataset (3.0 marks)

**Using the functions you have implemented above, please**

<b> 1. </b>
For each of the distance functions you implemented in Question 2, construct four KNN classifiers with K=1, K=5, K=25, K=55. You will create a total of 16 (4 distance functions x 4 K values) classifiers.

<b> 2. </b>
Compute the test accuracy for each model

In [115]:
########################
# Your code STARTS HERE
########################
#Beacuse of the note in Q2, each individual value of the numeric feature (legs) belongs constituting a separate class. 
#For calculating the distance, I convert feature (legs) into six features(0-legs/2-legs/4-legs/5-legs/6-legs/8-legs/).
#So there are 21 features now(16+6-1), 1 means true, 0 means false.
new_train_features = []
new_test_features = []

for train_feature in train_features:
    new_train_feature = []
    for i in range(len(train_feature)):
        if i != 12:
            new_train_feature.append(train_feature[i])
        elif train_feature[i] == 0:
            new_train_feature.extend([1, 0, 0, 0, 0, 0])
        elif train_feature[i] == 2:
            new_train_feature.extend([0, 1, 0, 0, 0, 0])
        elif train_feature[i] == 4:
            new_train_feature.extend([0, 0, 1, 0, 0, 0])
        elif train_feature[i] == 5:
            new_train_feature.extend([0, 0, 0, 1, 0, 0])
        elif train_feature[i] == 6:
            new_train_feature.extend([0, 0, 0, 0, 1, 0])
        elif train_feature[i] == 8:
            new_train_feature.extend([0, 0, 0, 0, 0, 1])
    new_train_features.append(new_train_feature)

for test_feature in test_features:
    new_test_feature = []
    for i in range(len(test_feature)):
        if i != 12:
            new_test_feature.append(test_feature[i])
        elif test_feature[i] == 0:
            new_test_feature.extend([1, 0, 0, 0, 0, 0])
        elif test_feature[i] == 2:
            new_test_feature.extend([0, 1, 0, 0, 0, 0])
        elif test_feature[i] == 4:
            new_test_feature.extend([0, 0, 1, 0, 0, 0])
        elif test_feature[i] == 5:
            new_test_feature.extend([0, 0, 0, 1, 0, 0])
        elif test_feature[i] == 6:
            new_test_feature.extend([0, 0, 0, 0, 1, 0])
        elif test_feature[i] == 8:
            new_test_feature.extend([0, 0, 0, 0, 0, 1])
    new_test_features.append(new_test_feature)

# 1. Predict test labels with each KNN classifier
knn_euc_1 = KNN(new_train_features, train_labels, new_test_features, 1, eucledian_distance)
knn_euc_5 = KNN(new_train_features, train_labels, new_test_features, 5, eucledian_distance)
knn_euc_25 = KNN(new_train_features, train_labels, new_test_features, 25, eucledian_distance)
knn_euc_55 = KNN(new_train_features, train_labels, new_test_features, 55, eucledian_distance)

knn_cos_1 = KNN(new_train_features, train_labels, new_test_features, 1, cosine_distance)
knn_cos_5 = KNN(new_train_features, train_labels, new_test_features, 5, cosine_distance)
knn_cos_25 = KNN(new_train_features, train_labels, new_test_features, 25, cosine_distance)
knn_cos_55 = KNN(new_train_features, train_labels, new_test_features, 55, cosine_distance)

knn_jac_1 = KNN(new_train_features, train_labels, new_test_features, 1, jaccard_distance)
knn_jac_5 = KNN(new_train_features, train_labels, new_test_features, 5, jaccard_distance)
knn_jac_25 = KNN(new_train_features, train_labels, new_test_features, 25, jaccard_distance)
knn_jac_55 = KNN(new_train_features, train_labels, new_test_features, 55, jaccard_distance)

knn_ham_1 = KNN(new_train_features, train_labels, new_test_features, 1, hamming_distance)
knn_ham_5 = KNN(new_train_features, train_labels, new_test_features, 5, hamming_distance)
knn_ham_25 = KNN(new_train_features, train_labels, new_test_features, 25, hamming_distance)
knn_ham_55 = KNN(new_train_features, train_labels, new_test_features, 55, hamming_distance)


# 2. Compute the accuracy scores 
accuracy_knn_euc_1 = accuracy(knn_euc_1, test_labels)
accuracy_knn_euc_5 = accuracy(knn_euc_5, test_labels)
accuracy_knn_euc_25 = accuracy(knn_euc_25, test_labels)
accuracy_knn_euc_55 = accuracy(knn_euc_55, test_labels)

accuracy_knn_cos_1 = accuracy(knn_cos_1, test_labels)
accuracy_knn_cos_5 = accuracy(knn_cos_5, test_labels)
accuracy_knn_cos_25 = accuracy(knn_cos_25, test_labels)
accuracy_knn_cos_55 = accuracy(knn_cos_55, test_labels)

accuracy_knn_jac_1 = accuracy(knn_jac_1, test_labels)
accuracy_knn_jac_5 = accuracy(knn_jac_5, test_labels)
accuracy_knn_jac_25 = accuracy(knn_jac_25, test_labels)
accuracy_knn_jac_55 = accuracy(knn_jac_55, test_labels)

accuracy_knn_ham_1 = accuracy(knn_ham_1, test_labels)
accuracy_knn_ham_5 = accuracy(knn_ham_5, test_labels)
accuracy_knn_ham_25 = accuracy(knn_ham_25, test_labels)
accuracy_knn_ham_55 = accuracy(knn_ham_55, test_labels)
########################
# Your code ENDS HERE
########################


print("Euclidean")
print(accuracy_knn_euc_1)
print(accuracy_knn_euc_5)
print(accuracy_knn_euc_25)
print(accuracy_knn_euc_55)
print("Cosine")
print(accuracy_knn_cos_1)
print(accuracy_knn_cos_5)
print(accuracy_knn_cos_25)
print(accuracy_knn_cos_55)
print("Jaccard")
print(accuracy_knn_jac_1)
print(accuracy_knn_jac_5)
print(accuracy_knn_jac_25)
print(accuracy_knn_jac_55)
print("Hamming")
print(accuracy_knn_ham_1)
print(accuracy_knn_ham_5)
print(accuracy_knn_ham_25)
print(accuracy_knn_ham_55)

Euclidean
0.8181818181818182
0.8181818181818182
0.7272727272727273
0.45454545454545453
Cosine
0.8181818181818182
0.8181818181818182
0.6363636363636364
0.45454545454545453
Jaccard
0.8181818181818182
0.7272727272727273
0.6363636363636364
0.45454545454545453
Hamming
0.8181818181818182
0.7272727272727273
0.7272727272727273
0.45454545454545453


### Question 6: Discussion (3.0 marks)
1. (a) Which parameter K resulted in the best performance? (b) Why? (c) What could be done to improve those classifiers that are currently performing poorly?

2. The results of KNN with Euclidean distance and KNN with Cosine distance are remarkably similar. Why is that so? Referring to the definitions of the distance functions.

<b>Each question should be answered in no more than 2-3 sentences.</b>

1(a).Overall, K = 1 resulted in the best performance. However, K = 5 also performs best based on some distance functions.

1(b).In this case, I think that when k=1, there is no need to deal with ties, but when k=5, the method of dealing with ties is random, it will lead to a different result each time. In addition, I think the number of test features is small, which results in slight difference in accuracy when k=1 and 5. For the larger K(25、55), the result will be classified into unrelated classes in a high risk, so the accuracy of K=25、55 is low.

1(c).When the neighbors disagree, we can deal with that by weighted KNN instead of majority voting. 

2.Euclidean distance reflects the absolute difference in value, while cosine distance reflects the difference in direction. When we normalize the feature vectors to unit vectors (assuming 2-dimensional vectors), we can found that Euclidean_distance = sqrt(2*Cosine_distance), so the two distance are remarkably similar.
